# Entropy Pooling example
This notebook replicates the results of the original EP method (Table 4 and Table 7) from the Sequential Entropy Pooling Heuristics article, available on https://ssrn.com/abstract=3936392.

In [1]:
import numpy as np
import fortitudo.tech as ft

In [2]:
# Load data 
R_df = ft.load_pnl()
instrument_names = R_df.columns

# Print prior P&L stats (Table 1 and Table 5)
stats_prior = ft.simulation_moments(R_df)
stats_prior.iloc[:, 0:2] = stats_prior.iloc[:, 0:2].mul(100)
display(np.round(stats_prior, 2))

corr_prior = ft.correlation_matrix(R_df)
display(np.round(100 * corr_prior))

,Mean,Volatility,Skewness,Kurtosis
Gov & MBS,-0.70,3.18,0.10,3.02
Corp IG,-0.40,3.39,0.11,3.11
Corp HY,1.92,6.12,0.17,2.97
EM Debt,2.74,7.51,0.22,3.06
DM Equity,6.40,14.85,0.40,3.15
EM Equity,8.00,26.87,0.77,4.10
Private Equity,13.70,27.77,0.72,3.76
Infrastructure,5.87,10.83,0.31,3.19
Real Estate,4.32,8.15,0.23,3.09
Hedge Funds,4.80,7.24,0.20,3.05


,0,1,2,3,4,5,6,7,8,9
"(0, Gov & MBS)",100.0,60.0,-0.0,30.0,-20.0,-10.0,-30.0,-10.0,-20.0,-20.0
"(1, Corp IG)",60.0,100.0,50.0,60.0,10.0,20.0,10.0,10.0,10.0,30.0
"(2, Corp HY)",-0.0,50.0,100.0,60.0,60.0,69.0,59.0,30.0,30.0,70.0
"(3, EM Debt)",30.0,60.0,60.0,100.0,40.0,59.0,30.0,20.0,20.0,40.0
"(4, DM Equity)",-20.0,10.0,60.0,40.0,100.0,69.0,79.0,40.0,40.0,80.0
"(5, EM Equity)",-10.0,20.0,69.0,59.0,69.0,100.0,69.0,30.0,39.0,79.0
"(6, Private Equity)",-30.0,10.0,59.0,30.0,79.0,69.0,100.0,39.0,49.0,79.0
"(7, Infrastructure)",-10.0,10.0,30.0,20.0,40.0,30.0,39.0,100.0,40.0,40.0
"(8, Real Estate)",-20.0,10.0,30.0,20.0,40.0,39.0,49.0,40.0,100.0,50.0
"(9, Hedge Funds)",-20.0,30.0,70.0,40.0,80.0,79.0,79.0,40.0,50.0,100.0


# Entropy Pooling views
We then specify the same views as in https://ssrn.com/abstract=3936392 : mean of Private Equity is 10%, volatility of EM Equity is less than or equal to 20%, skewness of DM Equity is less than or equal to −0.75, kurtosis of DM Equity is greater than or equal to 3.5, and correlation between Corp HY and EM Debt is 50%.

In [3]:
R = R_df.values
S = R.shape[0]
p = np.ones((S, 1)) / S
means_prior = p.T @ R
vols_prior = np.sqrt(p.T @ (R - means_prior)**2)

mean_rows = R[:, 2:7].T
vol_rows = (R[:, 2:6] - means_prior[:, 2:6]).T**2
skew_row = ((R[:, 4] - means_prior[:, 4]) / vols_prior[:, 4])**3
kurt_row = ((R[:, 4] - means_prior[:, 4]) / vols_prior[:, 4])**4
corr_row = (R[:, 2] - means_prior[:, 2]) * (R[:, 3] - means_prior[:, 3])

A = np.vstack((np.ones((1, S)), mean_rows, vol_rows[0:-1, :], corr_row[np.newaxis, :]))
b = np.vstack(([1], means_prior[:, 2:6].T, [0.1], vols_prior[:, 2:5].T**2,
               [0.5 * vols_prior[0, 2] * vols_prior[0, 3]]))
G = np.vstack((vol_rows[-1, :], skew_row, -kurt_row))
h = np.array([[0.2**2], [-0.75], [-3.5]])

# Posterior probabilities, relative entropy, and effective number of scenarios
Next, we calculate the posterior probabilities q, relative entropy (RE), and effective number of scenarios (ENS). Means, volatilities, skewness, kurtosis, and the correlation matrix are then
recalculated using the posterior probabilities. Finally, we print the posterior results (Table 4 and Table 7).

In [4]:
q = ft.entropy_pooling(p, A, b, G, h)
relative_entropy = q.T @ (np.log(q) - np.log(p))
effective_number_scenarios = np.exp(-relative_entropy)

In [5]:
stats_post = ft.simulation_moments(R_df, q)
stats_post.iloc[:, 0:2] = stats_post.iloc[:, 0:2].mul(100)
display(np.round(stats_post, 2))

print(f'ENS = {np.round(effective_number_scenarios[0, 0] * 100, 2)}%.')
print(f'RE = {np.round(relative_entropy[0, 0] * 100, 2)}%.')

corr_post = ft.correlation_matrix(R_df, q)
display(np.round(100 * corr_post))

,Mean,Volatility,Skewness,Kurtosis
Gov & MBS,-0.57,3.23,0.06,2.91
Corp IG,-0.51,3.43,0.14,3.12
Corp HY,1.92,6.12,-0.06,2.97
EM Debt,2.74,7.51,0.13,3.07
DM Equity,6.40,14.85,-0.75,3.50
EM Equity,8.00,20.00,-0.22,3.34
Private Equity,10.00,24.29,0.12,3.17
Infrastructure,5.65,10.65,0.28,3.16
Real Estate,3.75,8.02,0.13,3.02
Hedge Funds,4.64,6.95,-0.62,3.81


ENS = 70.92%.
RE = 34.36%.


,0,1,2,3,4,5,6,7,8,9
"(0, Gov & MBS)",100.0,60.0,-2.0,35.0,-23.0,-10.0,-34.0,-10.0,-20.0,-24.0
"(1, Corp IG)",60.0,100.0,51.0,63.0,9.0,20.0,7.0,9.0,11.0,29.0
"(2, Corp HY)",-2.0,51.0,100.0,50.0,57.0,64.0,55.0,27.0,27.0,67.0
"(3, EM Debt)",35.0,63.0,50.0,100.0,31.0,51.0,16.0,16.0,15.0,29.0
"(4, DM Equity)",-23.0,9.0,57.0,31.0,100.0,66.0,76.0,37.0,38.0,79.0
"(5, EM Equity)",-10.0,20.0,64.0,51.0,66.0,100.0,62.0,27.0,36.0,75.0
"(6, Private Equity)",-34.0,7.0,55.0,16.0,76.0,62.0,100.0,38.0,47.0,76.0
"(7, Infrastructure)",-10.0,9.0,27.0,16.0,37.0,27.0,38.0,100.0,39.0,38.0
"(8, Real Estate)",-20.0,11.0,27.0,15.0,38.0,36.0,47.0,39.0,100.0,49.0
"(9, Hedge Funds)",-24.0,29.0,67.0,29.0,79.0,75.0,76.0,38.0,49.0,100.0


# Comments
The results for the sequential heuristics in https://ssrn.com/abstract=3936392 are not replicated as they are a part of Fortitudo Technologies' proprietary software, which also contains an elegant interface for handling the different views instead of manually specifying them through A, b, G, and h. The interested reader can replicate the results of the sequential heuristics by using the P&L simulation that follows with this package and the Entropy Pooling technology.

# License

In [6]:
# fortitudo.tech - Novel Investment Technologies.
# Copyright (C) 2021-2023 Fortitudo Technologies.

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.